In [1]:
import torch
import numpy as np
from torchvision import datasets, transforms
from pathlib import Path
import spyrit.misc.walsh_hadamard as wh

from spyrit.misc.statistics import stat_walsh_stl10
from spyrit.misc.statistics import *
from spyrit.misc.disp import *

In [2]:
img_size = 64 # image size
M = 1024    # number of measurements
N0 = 10     # Image intensity (in photons)

#- Model and data paths
data_root = Path('../../data/')
stats_root = Path('../../data/stats_walsh')

In [3]:
#%% A batch of STL-10 test images
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(7)

transform = transforms.Compose(
    [transforms.functional.to_grayscale,
     transforms.Resize((img_size, img_size)),
     transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])

testset = \
    torchvision.datasets.STL10(root=data_root, split='test',download=False, transform=transform)
testloader =  torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

In [4]:
inputs, _ = next(iter(testloader))
b,c,h,w = inputs.shape

In [5]:
# stat_walsh_stl10()
Cov = np.load(stats_root / Path("Cov_{}x{}.npy".format(img_size, img_size)))
Mean = np.load(stats_root / Path("Average_{}x{}.npy".format(img_size, img_size)))
H =  wh.walsh2_matrix(img_size)
# H =  wh.walsh2_matrix(img_size)/img_size
# Cov /= img_size*img_size # THIS NEEDS TO BE NORMALIAZED FOR CONSISTENCY!


Ord = Cov2Var(Cov)
Perm = Permutation_Matrix(Ord)
Pmat = np.dot(Perm,H);
Pmat = Pmat[:M,:];


NameError: name 'Permutation_Matrix' is not defined

In [ ]:
x = inputs.view(b*c,w*h)

In [ ]:
img = x[1,:]
img = img.numpy();
imagesc(np.reshape(img,(h,w)))

In [ ]:
from spyrit.restructured.Updated_Had_Dcan import * 

# Forward Operator
## Instancier

In [ ]:
FO = Forward_operator(Pmat)

In [ ]:
print(FO.N)
print(FO.M)
print(FO.Hsub)
print(FO.Hsub_adjoint)

In [ ]:
print(FO.Hsub.weight.data)

In [ ]:
print(FO.Hsub_adjoint.weight.data)

Pour fadoua - check that same weigths as Pmat and Pmat.T
And compare two lines

# forward method

In [ ]:
y = FO(x)

In [ ]:
y.shape

Needs to be of shape (b*c, M)

## adjoint method

In [ ]:
x_est = FO.adjoint(y)

In [ ]:
x_est.shape

Needs to be of shape (b*c, N)

In [ ]:
img = 1/(w*h)*x_est[1,:]
img = img.numpy();
imagesc(np.reshape(img,(h,w)))

Close to initial image with downsample artifacts - OK

## Mat method

In [ ]:
print(FO.Mat())
print(FO.Mat().shape)

# Pos-neg split forward operator
## Instancier

In [ ]:
FO_split = Split_Forward_operator(Pmat)

In [ ]:
print(FO_split.N)
print(FO_split.M)
print(FO_split.Hsub)
print(FO_split.Hsub_adjoint)
print(FO_split.Hpos_neg)

In [ ]:
print(FO_split.Hpos_neg.weight.data)

Display a few lines as images

In [ ]:
m = FO_split((x+1)/2)

In [ ]:
m.shape

Should be of shape (bc, 2M)

In [ ]:
even_index = range(0,2*M,2);
odd_index = range(1,2*M,2);
mplus = m[:, even_index];
mminus = m[:, odd_index];
y_est = 2*(mplus-mminus) - FO(torch.ones_like(x))
print((y-y_est)/torch.norm(y))

Relative error in the $10^{-8}$s

# Split_Forward_operator_pylops 
## Instancier

In [ ]:
# FO_Pyl = Split_Forward_operator_pylops(Pmat)

Tests will happen with tikhonov regularisation

# Split_Forward_operator_ft_had
## Instancier

In [ ]:
FO_Had = Split_Forward_operator_ft_had(Pmat, Perm)

In [ ]:
Perm@Perm.T

In [ ]:
print(FO_Had.Perm)
print(FO_Had.Perm.weight.data)

In [ ]:
x_inv_tran = torch.zeros_like(x)
x_inv_tran[:,:M] = y
x_inv = FO_Had.inverse(x_inv_tran);

In [ ]:
img = x_inv[1,:]
img = img.numpy();
imagesc(np.reshape(img,(h,w)))

In [ ]:
# B = Bruit_Poisson_approx_Gauss(N0,F)

In [ ]:
# m_alpha = B(x)

In [ ]:
# m_alpha.shape

In [ ]:
# print(N0*m-m_alpha)

In [ ]:
# m

In [ ]:
# m_alpha

Ok, seems like Bruit_Poisson_approx_Gauss works as intended